<a href="https://colab.research.google.com/github/martinpius/RNN-ALIENS/blob/main/RNN_from_scratch_with_the_bidirectional_LSTM_architecture_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on Google CoLaB with Pytorch version {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)} {e}\n>>>> please correct {type(e)} and reload your drive")
  COLAB = False
def __time__(t: float = 123.918)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h} min: {m:>02} sec: {s:>05.2f}"
if torch.cuda.is_available():
  device = torch.device("cuda")
else: 
  device = torch.device("cpu")
print(f">>>> time formating\t.........................\n>>>> time elapsed\t{__time__()}")


Mounted at /content/drive
>>>> You are on Google CoLaB with Pytorch version 1.8.1+cu101
>>>> time formating	.........................
>>>> time elapsed	hrs: 0 min: 02 sec: 03.00


In [2]:
#In this notebook we are going to implimenting RNN with a Bi-directional LSTM architecture 
#Demonstration will be on MNIST dataset (batch, 1, 28, 28) by treating the width and length
#pixels as sequence length and input dimension. The channel dimension will be squeezed

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import time, sys, datetime

In [4]:
#Hyperparameters to be used in this network

In [5]:
batch_size = 128
hidden_dim = 512
num_layers = 2
num_classes = 10
learning_rate = 1e-3
seq_len = 28
input_dim = 28
EPOCHS = 10

In [6]:
#This class will build a bidirectional rnn with an lstm architecture:
class BRNN(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
    super(BRNN, self).__init__()
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers
    self.blstm = nn.LSTM(input_size = input_dim,
                         hidden_size = hidden_dim,
                         num_layers = num_layers,
                         batch_first = True,
                         dropout = 0.5,
                         bidirectional = True)
    self.fc1 = nn.Linear(in_features = 2*hidden_dim, out_features = 512)
    self.drp = nn.Dropout(p = 0.5)
    self.fc2 = nn.Linear(in_features = 512, out_features = 256)
    self.outputs = nn.Linear(in_features = 256, out_features = num_classes)
  
  def forward(self, input_tensor):
    h0 = torch.zeros(2*self.num_layers, input_tensor.size(0), self.hidden_dim).to(device = device) #initialize the previous hidden state to 0
    c0 = torch.zeros(2*self.num_layers, input_tensor.size(0), self.hidden_dim).to(device = device) #initialize previous cell state to 0
    out, _ = self.blstm(input_tensor, (h0, c0))
    out = out[:, -1, :] #select only the last layer hidden states
    x = self.fc1(out)
    x = self.drp(x)
    x = self.fc2(x)
    x = self.outputs(x)
    return x


In [7]:
#instantiating the model class and test using random data to see if the model gives the desired outcomes
rand_inputs = torch.rand(size = (64, 28, 28)).to(device = device)
model = BRNN(input_dim = input_dim,
             hidden_dim = hidden_dim,
             num_layers = num_layers,
             num_classes = num_classes).to(device = device)
print(f">>>> the output shape is: {model(rand_inputs).shape}")

>>>> the output shape is: torch.Size([64, 10])


In [8]:
#Get the data and pre-process it using the transform function, and splits into batches using the Loader method:
train_data = datasets.MNIST(root = "mnist_lstm_train/", train = True, transform = transforms.ToTensor(), download = True)
test_data = datasets.MNIST(root = "mnist_lstm_test/", train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, shuffle = True, batch_size = batch_size)
test_loader = DataLoader(dataset = test_data, shuffle = False, batch_size = batch_size)
x_train_batch, y_train_batch = next(iter(train_loader))
print(f"x_train_batch_shape: {x_train_batch.shape}\ty_train_batch_shape: {y_train_batch.shape}")

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_lstm_train/MNIST/raw/train-images-idx3-ubyte.gz to mnist_lstm_train/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_lstm_train/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_lstm_train/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_lstm_train/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_lstm_train/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_lstm_train/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_lstm_train/MNIST/raw

Processing...
Done!


...

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_lstm_test/MNIST/raw/train-images-idx3-ubyte.gz to mnist_lstm_test/MNIST/raw




Extracting mnist_lstm_test/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_lstm_test/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_lstm_test/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_lstm_test/MNIST/raw




Extracting mnist_lstm_test/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_lstm_test/MNIST/raw

Processing...
Done!
x_train_batch_shape: torch.Size([128, 1, 28, 28])	y_train_batch_shape: torch.Size([128])


In [9]:
#Getting the loss and an optimizer for the training loop:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)

In [10]:
#The training loop from scratch:
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>> training begins for epoch {epoch + 1}.............\n>>>> please wait while the model is training..............")
  for idx,(data, target) in enumerate(tqdm(train_loader)):
    data = data.squeeze(1).to(device = device)
    target = target.to(device = device)
    preds = model(data)
    train_loss = loss_fn(preds, target)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

def __tracking__(loader, model):
  if loader.dataset.train:
    print(f"\n>>>> checking performance on the training dataset\n>>>> please wait................")
  else:
    print(f"\n>>>> checking the performance on the validation dataset/n>>>> please wait..........")
  num_examples = 0
  num_correct = 0
  model.eval()
  with torch.no_grad():
    for idx, (x, y) in enumerate(tqdm(loader)):
      x = x.squeeze(1).to(device = device)
      y = y.to(device = device)
      preds = model(x)
      _, predictions = preds.max(1)
      num_correct+=(predictions == y).sum()
      num_examples+= predictions.size(0)
      acc = float((num_correct/num_examples) * 100)
      print(f"\n>>>> model is training for epoch {epoch + 1} of batch {idx}.......\n>>>> accuracy recorded: {acc:.4f}")
  model.train()
print(f">>>> Running with the training data: {__tracking__(train_loader, model)}")
print(f"\n>>>> Runnng with the validation data: {__tracking__(test_loader, model)}")
toc = time.time()
print(f"\ntime elapsed: {__time__(toc - tic)}")

  0%|          | 1/469 [00:00<00:59,  7.81it/s]


>>>> training begins for epoch 1.............
>>>> please wait while the model is training..............


  0%|          | 2/469 [00:00<00:23, 19.58it/s]


>>>> training begins for epoch 2.............
>>>> please wait while the model is training..............


  0%|          | 2/469 [00:00<00:24, 18.87it/s]

...


>>>> model is training for epoch 10 of batch 56.......
>>>> accuracy recorded: 98.4923

>>>> model is training for epoch 10 of batch 57.......
>>>> accuracy recorded: 98.5183

>>>> model is training for epoch 10 of batch 58.......
>>>> accuracy recorded: 98.5302

>>>> model is training for epoch 10 of batch 59.......
>>>> accuracy recorded: 98.5547

>>>> model is training for epoch 10 of batch 60.......
>>>> accuracy recorded: 98.5784

>>>> model is training for epoch 10 of batch 61.......
>>>> accuracy recorded: 98.5887

>>>> model is training for epoch 10 of batch 62.......
>>>> accuracy recorded: 98.6111

>>>> model is training for epoch 10 of batch 63.......
>>>> accuracy recorded: 98.6084

>>>> model is training for epoch 10 of batch 64.......
>>>> accuracy recorded: 98.6058


 89%|████████▊ | 70/79 [00:01<00:00, 40.68it/s]


>>>> model is training for epoch 10 of batch 65.......
>>>> accuracy recorded: 98.6151

>>>> model is training for epoch 10 of batch 66.......
>>>> accuracy recorded: 98.6357

>>>> model is training for epoch 10 of batch 67.......
>>>> accuracy recorded: 98.6558

>>>> model is training for epoch 10 of batch 68.......
>>>> accuracy recorded: 98.6753

>>>> model is training for epoch 10 of batch 69.......
>>>> accuracy recorded: 98.6830

>>>> model is training for epoch 10 of batch 70.......
>>>> accuracy recorded: 98.6576

>>>> model is training for epoch 10 of batch 71.......
>>>> accuracy recorded: 98.6654

>>>> model is training for epoch 10 of batch 72.......
>>>> accuracy recorded: 98.6836

>>>> model is training for epoch 10 of batch 73.......
>>>> accuracy recorded: 98.7014


100%|██████████| 79/79 [00:01<00:00, 40.33it/s]


>>>> model is training for epoch 10 of batch 74.......
>>>> accuracy recorded: 98.6979

>>>> model is training for epoch 10 of batch 75.......
>>>> accuracy recorded: 98.7048

>>>> model is training for epoch 10 of batch 76.......
>>>> accuracy recorded: 98.6810

>>>> model is training for epoch 10 of batch 77.......
>>>> accuracy recorded: 98.6979

>>>> model is training for epoch 10 of batch 78.......
>>>> accuracy recorded: 98.7000

>>>> Runnng with the validation data: None

time elapsed: hrs: 0 min: 04 sec: 57.00
